In [16]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from termcolor import colored
from PIL import Image
import dlib
#import cv2
from cv2 import resize
detector = dlib.get_frontal_face_detector()
#face_cascade = cv2.CascadeClassifier('/home/mckc/Downloads/opencv-2.4.13/data/haarcascades_GPU/haarcascade_frontalface_default.xml')

In [17]:
import os
os.chdir('/home/mckc/Downloads/faces/train//')

In [18]:
train = pd.read_csv('/home/mckc/All Data///train.csv')
test = pd.read_csv('/home/mckc/All Data///test.csv')

print 'the training data shape is ',train.shape
print 'the test data shape is ', test.shape
train_records = train.shape[0]
test_records = test.shape[0]
    
X_tr = np.zeros((1,3,224,224),dtype=np.uint8)
Y_tr =[]
iteration = 0
for i in train.values[:,0]:
    image = np.array(Image.open(i),dtype=np.uint8)
    faces = detector(image, 1)
#    faces = face_cascade.detectMultiScale(
#        cv2.cvtColor(image, cv2.COLOR_BGR2GRAY),
#        scaleFactor=1.1,
#        minNeighbors=5,
#        minSize=(30, 30),
#       flags=cv2.cv.CV_HAAR_SCALE_IMAGE
#    )
    if len(faces)==1:    
        for a,b in enumerate(faces):
#        for (x, y, w, h) in faces:
#            fac = resize(np.array(image)[y:(y+h),x:(x+h),:] ,(3,224,224)).reshape(-1,3,224,224)
            fac = resize(np.array(image)[b.top():b.bottom(),b.left():b.right(),:] ,(224,224)).astype(np.float32)
        #fac[:,:,0] -= 103.939
        #fac[:,:,1] -= 116.779
        #fac[:,:,2] -= 123.68
        fac = fac.transpose((2,0,1))
        scipy.misc.toimage(fac).save(train.values[iteration,1]+'_'+str(iteration) +'.jpg')
        fac = np.expand_dims(fac, axis=0)
        
        X_tr =  np.vstack((X_tr,fac))
        Y_tr = np.append(Y_tr,train.values[iteration,1])
    iteration+=1
    if iteration % 50==0:
        print colored((float(iteration)/train_records*100 ,' Percentage complete'), 'green')
    

X_tr = X_tr[1:,:,:]
print 'the training file shape',X_tr.shape,Y_tr.shape
os.chdir('/home/mckc/Downloads/faces/test/')

iteration = 0      
X_ts = np.zeros((1,3,224,224),dtype=np.uint8)
Y_ts = []
for i in test.values[:,0]:
    image = np.array(Image.open(i),dtype=np.uint8)
    faces = detector(image, 1)
#    faces = face_cascade.detectMultiScale(
#        cv2.cvtColor(image, cv2.COLOR_BGR2GRAY),
#        scaleFactor=1.1,
#        minNeighbors=5,
#        minSize=(30, 30),
#       flags=cv2.cv.CV_HAAR_SCALE_IMAGE
#    )
    if len(faces)==1:    
        for a,b in enumerate(faces):
#        for (x, y, w, h) in faces:
#            fac = resize(np.array(image)[y:(y+h),x:(x+h),:] ,(3,224,224)).reshape(-1,3,224,224)
            fac = resize(np.array(image)[b.top():b.bottom(),b.left():b.right(),:] ,(224,224)).astype(np.float32)
        #fac[:,:,0] -= 103.939
        #fac[:,:,1] -= 116.779
        #fac[:,:,2] -= 123.68
        fac = fac.transpose((2,0,1))
        scipy.misc.toimage(fac).save(test.values[iteration,1]+'_'+str(iteration) +'.jpg')
        fac = np.expand_dims(fac, axis=0)
        X_ts =  np.vstack((X_ts,fac))
        Y_ts = np.append(Y_ts,test.values[iteration,1])
    iteration+=1
    if iteration % 50==0:
        print colored((float(iteration)/test_records*100 ,' Percentage complete'), 'green')
    
X_ts = X_ts[1:,:,:]

print 'the testing file shape',X_ts.shape,Y_ts.shape

the training data shape is  (586, 2)
the test data shape is  (195, 2)
(8.532423208191126, ' Percentage complete')
(17.064846416382252, ' Percentage complete')
(25.597269624573375, ' Percentage complete')
(34.129692832764505, ' Percentage complete')
(42.66211604095563, ' Percentage complete')
(51.19453924914675, ' Percentage complete')
(59.72696245733788, ' Percentage complete')
(68.25938566552901, ' Percentage complete')
(76.79180887372013, ' Percentage complete')
(85.32423208191126, ' Percentage complete')
(93.85665529010238, ' Percentage complete')
the training file shape (586, 3, 224, 224) (586,)
(25.64102564102564, ' Percentage complete')
(51.28205128205128, ' Percentage complete')
(76.92307692307693, ' Percentage complete')
the testing file shape (193, 3, 224, 224) (193,)


In [ ]:
#from keras.datasets import cifar10
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
#(X_train, y_train), (X_test, y_test) = cifar10.load_data()
map, Y_number = np.unique(Y_tr, return_inverse=True)
Y_test_number = np.unique(Y_ts, return_inverse=True)[1]

Y_train = np_utils.to_categorical(Y_number, 7)
Y_test = np_utils.to_categorical(Y_test_number, 7)

datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    )

# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(X_tr.astype(np.float32))

In [ ]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD,Adagrad

def VGG_16(weights_path=None):
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(3,224,224)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1000, activation='softmax'))

    if weights_path:
        model.load_weights(weights_path)

    return model

model = VGG_16('/home/mckc/Downloads/vgg16_weights.h5')

model.layers.pop()
model.add(Dense(7, activation='softmax'))

sgd = SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)

adagrad = Adagrad(lr=0.001, epsilon=1e-08)
model.compile(optimizer=sgd, loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(X_tr.astype(np.float16),Y_train,verbose=1,validation_data=(X_ts.astype(np.float16),Y_test),nb_epoch=100,batch_size=2)

In [ ]:
# fits the model on batches with real-time data augmentation:
model.fit_generator(datagen.flow(X_tr, Y_train, batch_size=10),validation_data=(X_ts.astype(np.float16),Y_test)
                    ,nb_worker=7,samples_per_epoch=len(X_tr), nb_epoch=1000, pickle_safe=True)

In [ ]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD,Adagrad
import cv2, numpy as np

def VGG_19(weights_path=None):
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(3,224,224)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1000, activation='softmax'))

    if weights_path:
        model.load_weights(weights_path)

    return model


model = VGG_19('/home/mckc/Downloads/vgg19_weights.h5')

model.layers.pop()
model.add(Dense(2, activation='softmax'))

sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
adagrad = Adagrad(lr=0.001, epsilon=1e-08)
model.compile(optimizer=sgd, loss='categorical_crossentropy')

In [ ]:
model.fit(X_tr.astype(np.float16),Y_train,verbose=1,validation_data=(X_ts.astype(np.float16),Y_test)
          ,nb_epoch=1000,batch_size=2)

In [ ]:
# fits the model on batches with real-time data augmentation:
model.fit_generator(datagen.flow(X_tr, Y_train, batch_size=10),validation_data=(X_ts.astype(np.float16),Y_test)
                    ,nb_worker=7,samples_per_epoch=len(X_tr), nb_epoch=1000, pickle_safe=True)